In [1]:
from jetbot import ObjectDetector
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [2]:
from jetbot import Camera
camera = Camera.instance(width=300, height=300)

In [ ]:
detections = model(camera.value)
print(detections)

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()
detections_widget.value = str(detections)
display(detections_widget)

In [ ]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
from jetbot import Robot
robot = Robot()

In [ ]:
import PID

global follow_speed
follow_speed = 0.5
global turn_gain
turn_gain = 1.7
global follow_speed_pid, follow_speed_pid_model
follow_speed_pid_model = 1
follow_speed_pid = PID.PositionalPID(1.5, 0, 0.05)
global turn_gain_pid
turn_gain_pid = PID.PositionalPID(0.15, 0, 0.05)

In [ ]:
from jetbot import bgr8_to_jpeg

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=77, description='tracked label')

display(widgets.VBox([
    widgets.HBox([image_widget]),
    label_widget,
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Calculate the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Calculate the length of a two-dimensional vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Find the detection closest to the center of the image"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    global follow_speed
    global turn_gain
    global follow_speed_pid
    target_value_speed = 0
    image = change['new']

    detections = model(image)
    # mark the non-target objects with blue lines.
    # Draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (255, 0, 0), 2)
    # Select the object we want to track, Which is 17 -- cat
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # mark the target object with green lines.
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (0, 255, 0), 4)
    if det is None:
        robot.stop()
    else:
        # move robot forward and steer based on the distance between center of camera and center of target object
        center = detection_center(det)

        #Follow speed PID adjustment
        follow_speed_pid.SystemOutput = 90000 * (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
        if label_widget.value == 17:
            follow_speed_pid.SetStepSignal(30000)
        else:
            follow_speed_pid.SetStepSignal(10000)
        follow_speed_pid.SetInertiaTime(0.2, 0.1)

        #Steering gain PID adjustment
        turn_gain_pid.SystemOutput = center[0]
        turn_gain_pid.SetStepSignal(0)
        turn_gain_pid.SetInertiaTime(0.2, 0.1)

        #Limit steering gain to the effective range
        if label_widget.value == 17:
            target_value_turn_gain = 0.7+ abs(turn_gain_pid.SystemOutput)
        else:
            target_value_turn_gain = 0.4 abs(turn_gain_pid.SystemOutput)
        if target_value_turn_gain < 0:
            target_value_turn_gain = 0
        elif target_value_turn_gain > 2:
            target_value_turn_gain = 2

        #Keep the output motor speed within the effective driving range
        target_value_speed = 0.4 + follow_speed_pid.SystemOutput / 90000
        target_value_speedl = target_value_speed + target_value_turn_gain * center[0]
        target_value_speedr = target_value_speed - target_value_turn_gain * center[0]
        if target_value_speedl<0.5:
            target_value_speedl=0
        elif target_value_speedl>1:
            target_value_speedl = 1
        if target_value_speedr<0.5:
            target_value_speedr=0
        elif target_value_speedr>1:
            target_value_speedr = 1

        robot.set_motors(target_value_speedl, target_value_speedr)
        
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

In [ ]:
#camera.unobserve_all()
#camera.observe(execute, names='value')

import threading
import inspect
import ctypes
''' definition of the close thread function'''
def _async_raise(tid, exctype):
  """raises the exception, performs cleanup if needed"""
  tid = ctypes.c_long(tid)
  if not inspect.isclass(exctype):
    exctype = type(exctype)
  res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
  if res == 0:
    raise ValueError("invalid thread id")
  elif res != 1:
    ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
    raise SystemError("PyThreadState_SetAsyncExc failed")
def stop_thread(thread):
  _async_raise(thread.ident, SystemExit)
'''The function of thread 1, which continuously calls the detection function'''
def test():
    while True:
        execute({'new': camera.value})        

thread1 = threading.Thread(target=test)
thread1.setDaemon(False)
thread1.start()


In [ ]:
robot.stop()